In [1]:
import pandas as pd

# to split the datasets
from sklearn.model_selection import train_test_split

# to encode with feature-engine
from feature_engine.categorical_encoders import CountFrequencyCategoricalEncoder

In [2]:
data = pd.read_csv('creditApprovalUCI.csv')

data.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,202.0,0,1
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,43.0,560,1
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,280.0,824,1
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,100.0,3,1
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,120.0,0,1


In [3]:
# let's separate into training and testing set

X_train, X_test, y_train, y_test = train_test_split(
    data.drop(labels=['A16'], axis=1),  # predictors
    data['A16'],  # target
    test_size=0.3,  # percentage of obs in test set
    random_state=0)  # seed to ensure reproducibility

X_train.shape, X_test.shape

((483, 15), (207, 15))

## Count encoding with pandas

In [4]:
# let's obtain the counts for each category in the variable A7

count_map = X_train['A7'].value_counts().to_dict()

count_map

{'v': 277,
 'h': 101,
 'ff': 41,
 'bb': 39,
 'z': 7,
 'dd': 5,
 'j': 5,
 'Missing': 4,
 'n': 3,
 'o': 1}

In [5]:
# replace the labels with the counts

X_train['A7'] = X_train['A7'].map(count_map)
X_test['A7'] = X_test['A7'].map(count_map)

In [6]:
# let's explore the result

X_train['A7'].head(10)

596    277
303    277
204    277
351     41
118    277
247    101
652    277
513      5
230    277
250      7
Name: A7, dtype: int64

In [7]:
# if instead of the count we would like the frequency
# we need only divide the count by the total number of observations:

frequency_map = (X_train['A6'].value_counts() / len(X_train) ).to_dict()
frequency_map

{'c': 0.19254658385093168,
 'q': 0.11594202898550725,
 'w': 0.09937888198757763,
 'i': 0.08488612836438923,
 'k': 0.07867494824016563,
 'ff': 0.07867494824016563,
 'aa': 0.07039337474120083,
 'cc': 0.062111801242236024,
 'm': 0.053830227743271224,
 'x': 0.049689440993788817,
 'e': 0.043478260869565216,
 'd': 0.043478260869565216,
 'j': 0.016563146997929608,
 'Missing': 0.008281573498964804,
 'r': 0.002070393374741201}

In [8]:
# replace the labels with the frequency

X_train['A6'] = X_train['A6'].map(frequency_map)
X_test['A6'] = X_test['A6'].map(frequency_map)

### Putting the code into functions to smooth the process

In [9]:
def count_mappings(df, variable):
    return df[variable].value_counts().to_dict()


def frequency_mappings(df, variable):
    return (df[variable].value_counts() / len(df)).to_dict()


def encode(train, test, variable, mapping):
    X_train[variable] = X_train[variable].map(mapping)
    X_test[variable] = X_test[variable].map(mapping)

In [10]:
# make a list with the categorical variables

vars_categorical = ['A1', 'A4', 'A5', 'A6', 'A7', 'A9', 'A10', 'A12', 'A13']

In [11]:
# encode multiple categorical variables with the functions
# above

for variable in vars_categorical:
    mappings = count_mappings(X_train, variable)
    encode(X_train, X_test, variable, mappings)

## Count and frequency encoding with Feature-Engine

In [12]:
# let's divide into train and test sets (again)

X_train, X_test, y_train, y_test = train_test_split(
    data.drop(labels=['A16'], axis=1),  # predictors
    data['A16'],  # target
    test_size=0.3,  # percentage of obs in test set
    random_state=0)  # seed to ensure reproducibility

In [13]:
# let's create an encoder

count_enc = CountFrequencyCategoricalEncoder(
    encoding_method='count', # to do frequency ==> encoding_method='frequency'
    variables=None)

In [14]:
# fit the encoder to the train set
count_enc.fit(X_train)

CountFrequencyCategoricalEncoder(encoding_method='count',
                                 variables=['A1', 'A4', 'A5', 'A6', 'A7', 'A9',
                                            'A10', 'A12', 'A13'])

In [15]:
# the encoder stores the variables it will encode

count_enc.variables

['A1', 'A4', 'A5', 'A6', 'A7', 'A9', 'A10', 'A12', 'A13']

In [16]:
# in the encoder dict contains the counts or frequencies
# per category per variable

count_enc.encoder_dict_

{'A1': {'b': 335, 'a': 144, 'Missing': 4},
 'A4': {'u': 363, 'y': 115, 'Missing': 4, 'l': 1},
 'A5': {'g': 363, 'p': 115, 'Missing': 4, 'gg': 1},
 'A6': {'c': 93,
  'q': 56,
  'w': 48,
  'i': 41,
  'k': 38,
  'ff': 38,
  'aa': 34,
  'cc': 30,
  'm': 26,
  'x': 24,
  'e': 21,
  'd': 21,
  'j': 8,
  'Missing': 4,
  'r': 1},
 'A7': {'v': 277,
  'h': 101,
  'ff': 41,
  'bb': 39,
  'z': 7,
  'dd': 5,
  'j': 5,
  'Missing': 4,
  'n': 3,
  'o': 1},
 'A9': {'t': 256, 'f': 227},
 'A10': {'f': 271, 't': 212},
 'A12': {'f': 263, 't': 220},
 'A13': {'g': 441, 's': 38, 'p': 4}}

In [17]:
# let's transform train and test sets

X_train_enc = count_enc.transform(X_train)
X_test_enc = count_enc.transform(X_test)

C:\Users\Sole\Anaconda3\envs\feml\lib\site-packages\sklearn\utils\validation.py:933: FutureWarning: Passing attributes to check_is_fitted is deprecated and will be removed in 0.23. The attributes argument is ignored.
  "argument is ignored.", FutureWarning)


In [18]:
# let's inspect the result

X_train_enc.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15
596,144,46.08,3.000,363,363,93,277,2.375,256,212,8,220,441,396.0,4159
303,144,15.92,2.875,363,363,56,277,0.085,227,271,0,263,441,120.0,0
204,335,36.33,2.125,115,115,48,277,0.085,256,212,1,263,441,50.0,1187
351,335,22.17,0.585,115,115,38,41,0.000,227,271,0,263,441,100.0,0
118,335,57.83,7.040,363,363,26,277,14.000,256,212,6,220,441,360.0,1332


In [19]:
X_test_enc.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15
14,144,45.83,10.50,363,363,56,277,5.000,256,212,7,220,441,0.0,0
586,335,64.08,20.00,363,363,24,101,17.500,256,212,9,220,441,0.0,1000
140,144,31.25,3.75,363,363,30,101,0.625,256,212,9,220,441,181.0,0
492,335,39.25,9.50,363,363,26,277,6.500,256,212,14,263,441,240.0,4607
350,144,26.17,2.00,363,363,8,5,0.000,227,271,0,220,441,276.0,1
